In [109]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import recall_score


In [78]:
from numpy import genfromtxt
PCA_arr = genfromtxt('PCA_origin.csv', delimiter=',')

In [49]:
def normalizer_df(df, return_df = False, is_standard = True, range_to_put = (-1, 1)):
    df = df.copy()
    st_x= StandardScaler()
    norm_x = MinMaxScaler(feature_range = range_to_put)
    if is_standard:
        if return_df:
            df[df.columns] = st_x.fit_transform(df[df.columns])
            return df
        else:
            return st_x.fit_transform(df)
    else:
        if return_df:
            df[df.columns] = norm_x.fit_transform(df[df.columns])
            return df
        else:
            return norm_x.fit_transform(df)

In [20]:
train_df=pd.read_csv('train.csv')

,SITE_ID,DATE,CELL_TYPE_Macro,CELL_TYPE_Mobil,CELL_TYPE_TRP,CELL_TYPE_Tx site,CELL_TYPE_micro,N_TRANSPORTED_SITES,GEOGRAPHIC_CLUSTER_K_0,GEOGRAPHIC_CLUSTER_K_1,...,skew_equipment_alarms_prev14d,skew_fire/smoke_alarms_prev14d,skew_ge_alarms_prev14d,skew_power_alarms_prev14d,skew_temperature_alarms_prev14d,kurt_equipment_alarms_prev14d,kurt_fire/smoke_alarms_prev14d,kurt_ge_alarms_prev14d,kurt_power_alarms_prev14d,kurt_temperature_alarms_prev14d
0,146,2019-04-10,1,0,0,0,0,3.0,0,0,...,0.0,0.0,0.0,0.0,0.0,-1.212308,-1.212308,-1.212308,-1.212308,-1.212308
1,146,2019-04-11,1,0,0,0,0,3.0,0,0,...,0.0,0.0,0.0,0.0,0.0,-1.212308,-1.212308,-1.212308,-1.212308,-1.212308
2,146,2019-04-12,1,0,0,0,0,3.0,0,0,...,0.0,0.0,0.0,0.0,0.0,-1.212308,-1.212308,-1.212308,-1.212308,-1.212308
3,146,2019-04-13,1,0,0,0,0,3.0,0,0,...,0.0,0.0,0.0,0.0,0.0,-1.212308,-1.212308,-1.212308,-1.212308,-1.212308
4,146,2019-04-14,1,0,0,0,0,3.0,0,0,...,0.0,0.0,0.0,0.0,0.0,-1.212308,-1.212308,-1.212308,-1.212308,-1.212308
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
621295,1251,2020-01-30,1,0,0,0,0,7.0,0,0,...,0.0,0.0,0.0,0.0,0.0,-1.212308,-1.212308,-1.212308,-1.210000,-1.212308
621296,1251,2020-01-31,1,0,0,0,0,7.0,0,0,...,0.0,0.0,0.0,0.0,0.0,-1.212308,-1.212308,-1.212308,-1.210000,-1.212308
621297,1251,2020-02-01,1,0,0,0,0,7.0,0,0,...,0.0,0.0,0.0,0.0,0.0,-1.212308,-1.212308,-1.212308,-1.210000,-1.212308
621298,1251,2020-02-02,1,0,0,0,0,7.0,0,0,...,0.0,0.0,0.0,0.0,0.0,-1.212308,-1.212308,-1.212308,-1.210000,-1.212308


In [79]:
# Data Frame after PCA is implememnted
X=pd.DataFrame(PCA_arr)

In [32]:
# target Variable
target_variable = 'aircon_sum_target_next14d'
y=train_df['aircon_sum_target_next14d']

0         0
1         0
2         0
3         0
4         0
         ..
621295    0
621296    0
621297    0
621298    0
621299    0
Name: aircon_sum_target_next14d, Length: 621300, dtype: int64

Having the X and  in DataFrame platform, we split the data into train and test data set.

.

# Splitting data into Test and Train Set

In [31]:
count_class_0, count_class_1 = y.value_counts()
print(count_class_0, count_class_1)

617717 3583


In [80]:
concat_df= pd.concat([X,y], axis = 1)

In [81]:
from sklearn.utils import resample

#Over-sample Minority Class
#1) Separate majority and minority classes
df_majority = concat_df[concat_df[target_variable]==0] #"target" is the name of the target column, change it accordingly to your dataset
df_minority = concat_df[concat_df[target_variable]==1] #"target" is the name of the target column, change it accordingly to your dataset
 
#2) Oversample minority class
df_minority_oversampled = resample(df_minority, 
                                   replace=True,
                                   n_samples=100000,     # number of samples into the minority class (change it accordingly to your dataset)
                                   random_state=123)  # reproducible results
 
#3) Combine oversampled minority class with majority class
df_oversampled = pd.concat([df_minority_oversampled, df_majority])
 
#4) Display new class counts
df_oversampled[target_variable].value_counts() #"target" is the name of the target column, change it accordingly to your dataset

0    617717
1    100000
Name: aircon_sum_target_next14d, dtype: int64

In [82]:
from sklearn.model_selection import train_test_split
target_variable = 'aircon_sum_target_next14d'
input_variables = df_oversampled.columns[df_oversampled.columns!=target_variable]
X_origin_df = df_oversampled[input_variables]
y_origin_df = df_oversampled[target_variable]
X_origin_df = normalizer_df(X_origin_df, True)
#SPLIT DATA INTO TRAIN AND TEST SET
X_train, X_test, y_train, y_test = train_test_split(X_origin_df, y_origin_df, 
                                                    test_size =0.20, #by default is 75%-25%
                                                    random_state= 123) #fix random seed for replicability

print(X_train.shape, X_test.shape)

(574173, 32) (143544, 32)


# Models

In [41]:
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
from sklearn import metrics 
from sklearn.metrics import f1_score

from sklearn.metrics import precision_recall_fscore_support

# GreadSearch Algorithm Search 

In [42]:
def hyperp_search(classifier, parameters):
    gs = GridSearchCV(classifier, parameters, cv=3, scoring = 'f1', verbose=0, n_jobs=-1)
    gs = gs.fit(X_train, y_train)
    print("f1_train: %f using %s" % (gs.best_score_, gs.best_params_))

    best_model = gs.best_estimator_
    y_pred = best_model.predict(X_test)

    print("f1_test: ", f1_score(y_test, y_pred))
    print(confusion_matrix(y_test, y_pred))
    print(classification_report(y_test, y_pred))
    print(precision_recall_fscore_support(y_test, y_pred, average='weighted'))

# Random Forest

In [117]:
from sklearn.ensemble import RandomForestClassifier
classifier = RandomForestClassifier()
parameters = {'n_estimators':[10,50,100], "criterion":  ['entropy','gini'],
              'min_samples_leaf': [10,20,100], 'random_state' : [42], 'max_depth':[5,10,15,20]}


hyperp_search(classifier,parameters)

f1_train: 0.520193 using {'criterion': 'gini', 'max_depth': 10, 'min_samples_leaf': 5, 'n_estimators': 5, 'random_state': 42}
f1_test:  0.5391799790847788
[[123289    384]
 [ 12395   7476]]
              precision    recall  f1-score   support

           0       0.91      1.00      0.95    123673
           1       0.95      0.38      0.54     19871

    accuracy                           0.91    143544
   macro avg       0.93      0.69      0.74    143544
weighted avg       0.91      0.91      0.89    143544

(0.9145309556885896, 0.9109750320459232, 0.8937569489360926, None)


# Decision Tree

In [118]:
from sklearn.tree import DecisionTreeClassifier
classifier = DecisionTreeClassifier()
parameters = {'criterion': ['entropy','gini'], 
              'max_depth': [4,5,10],
              'min_samples_split': [20],
              'min_samples_leaf': [10]}

hyperp_search(classifier,parameters)

f1_train: 0.556077 using {'criterion': 'gini', 'max_depth': 10, 'min_samples_leaf': 10, 'min_samples_split': 20}
f1_test:  0.5703818336860057
[[120911   2762]
 [ 10841   9030]]
              precision    recall  f1-score   support

           0       0.92      0.98      0.95    123673
           1       0.77      0.45      0.57     19871

    accuracy                           0.91    143544
   macro avg       0.84      0.72      0.76    143544
weighted avg       0.90      0.91      0.89    143544

(0.8966829040497052, 0.9052346318898735, 0.8946433594745302, None)


# KNN

In [ ]:
from sklearn.neighbors import KNeighborsClassifier
classifier = KNeighborsClassifier()
parameters = {'n_neighbors':np.arange(10,1000,100)}

hyperp_search(classifier,parameters)

# Naive Bayes

In [53]:
from sklearn.naive_bayes import GaussianNB #or alternative NB implementations

model = GaussianNB()

model.fit(X_train, y_train)
y_pred=model.predict(X_test)

from sklearn.metrics import f1_score
print("f1_score: ", f1_score(y_test, y_pred))

print("f1_test: ", f1_score(y_test, y_pred))


print(precision_recall_fscore_support(y_test, y_pred, average='weighted'))

f1_score:  0.38051432934209795
f1_test:  0.38051432934209795
(0.8275647575974502, 0.7682452767095803, 0.7914356698947307, None)


# AdaBoost Classifier

In [68]:
from sklearn.ensemble import AdaBoostClassifier
classifier = AdaBoostClassifier()
parameters = {'base_estimator': [DecisionTreeClassifier(max_depth=3)], 'n_estimators':[50,10], 'learning_rate':[1.0,0.3,0.01]}
# AdaBoostClassifier(DecisionTreeClassifier(max_depth=3)
# base_estimator=DecisionTreeRegressor(max_depth=3)        !!!!!!!!!!


hyperp_search(classifier,parameters)

f1_train: 0.527312 using {'base_estimator': DecisionTreeClassifier(max_depth=3), 'learning_rate': 1.0, 'n_estimators': 50}
f1_test:  0.5270991647302644
[[120091   3582]
 [ 11478   8393]]
              precision    recall  f1-score   support

           0       0.91      0.97      0.94    123673
           1       0.70      0.42      0.53     19871

    accuracy                           0.90    143544
   macro avg       0.81      0.70      0.73    143544
weighted avg       0.88      0.90      0.88    143544

(0.8834292320757039, 0.8950844340411303, 0.8837006818119572, None)


# Bagging Classifier


In [70]:
from sklearn.ensemble import BaggingClassifier
classifier = BaggingClassifier()
parameters = {'base_estimator': [DecisionTreeClassifier(max_depth=3)], 'n_estimators':[50,10],'bootstrap':[True, False]}

hyperp_search(classifier,parameters)

f1_train: 0.220833 using {'base_estimator': DecisionTreeClassifier(max_depth=3), 'bootstrap': True, 'n_estimators': 50}
f1_test:  0.2455111762550385
[[121998   1675]
 [ 16856   3015]]
              precision    recall  f1-score   support

           0       0.88      0.99      0.93    123673
           1       0.64      0.15      0.25     19871

    accuracy                           0.87    143544
   macro avg       0.76      0.57      0.59    143544
weighted avg       0.85      0.87      0.83    143544

(0.8459712140931352, 0.8709036950342752, 0.8347394763916686, None)


# Gradient Boosting Classifier

In [121]:
from sklearn.ensemble import GradientBoostingClassifier

classifier = GradientBoostingClassifier()            # base_estimator=DecisionTreeRegressor(max_depth=3)
parameters = {'loss':['deviance'], 'learning_rate':[0.1,0.3],'n_estimators': [100], 'subsample':[1.0], 'criterion':['friedman_mse'], 'min_samples_split':[100, 50], 'min_samples_leaf':[10000], 'max_depth':[6,5,4,3], 'max_features':['auto']}

hyperp_search(classifier,parameters)

KeyboardInterrupt: 

In [75]:
regressor.get_params().keys()

dict_keys(['alpha', 'ccp_alpha', 'criterion', 'init', 'learning_rate', 'loss', 'max_depth', 'max_features', 'max_leaf_nodes', 'min_impurity_decrease', 'min_impurity_split', 'min_samples_leaf', 'min_samples_split', 'min_weight_fraction_leaf', 'n_estimators', 'n_iter_no_change', 'presort', 'random_state', 'subsample', 'tol', 'validation_fraction', 'verbose', 'warm_start'])

# Logistic Regression

In [55]:
from sklearn.linear_model import LogisticRegression

classifier = LogisticRegression()
parameters = {"C":[1e-4,1e-3,1e-2,1e-1,1,10], "max_iter":[1000] }


hyperp_search(classifier,parameters)

f1_train: 0.313322 using {'C': 10, 'max_iter': 1000}
f1_test:  0.31203940886699505
[[122128   1545]
 [ 15912   3959]]
              precision    recall  f1-score   support

           0       0.88      0.99      0.93    123673
           1       0.72      0.20      0.31     19871

    accuracy                           0.88    143544
   macro avg       0.80      0.59      0.62    143544
weighted avg       0.86      0.88      0.85    143544

(0.861827799401135, 0.8783857214512624, 0.8472955674547159, None)


# Making the Prediction

Having the model parameters and best fit model to our Train.csv Data, we will be able to predict over the Test.csv.
To be noticed, same scaled fitted model (standard Scalar) which has already implemented over the Train Data should be dedicated
toward the Test Test.

In [83]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
import datetime

In [84]:
from numpy import genfromtxt
PCA_arr = genfromtxt('PCA_test.csv', delimiter=',')

In [85]:
test_origin = pd.read_csv('test.csv')

Having the model parameters and best fit model to our Train.csv Data, we will be able to predict over the Test.csv.
To be noticed, same scaled fitted model (standard Scalar) which has already implemented over the Train Data should be dedicated
toward the Test Test.

In [86]:
X_pca_test=pd.DataFrame(PCA_arr)
test_origin = test_origin[['SITE_ID','DATE','N_TRANSPORTED_SITES']]
# we should do all the preprocessing that we did for train.csv on the test.csv as well, The result is in the X_2 data frame

In [87]:
X.head()

,0,1,2,3,4,5,6,7,8,9,...,22,23,24,25,26,27,28,29,30,31
0,-0.638963,-0.723192,1.073850,-0.020673,0.173960,-0.986872,-0.044315,-0.073762,0.318885,0.127459,...,0.228251,0.009626,0.024679,0.137854,0.068709,-0.048554,0.006412,0.025030,-0.178321,0.071160
1,-0.643231,-0.908548,1.280059,-0.098624,0.204755,-0.929311,-0.024817,-0.090516,0.362124,0.168404,...,0.249787,-0.011441,0.019608,0.166461,0.087784,-0.069482,-0.005775,0.014163,0.119573,0.014776
2,-0.630286,-1.024725,1.449132,-0.108238,0.159373,-0.975164,-0.014220,-0.097865,0.362856,0.169781,...,0.255645,-0.002174,0.017529,0.154766,0.077265,-0.049503,-0.007906,0.015206,0.299571,-0.017298
3,-0.615162,-0.894503,1.380044,-0.074874,0.003987,-1.386995,-0.097885,-0.032074,0.442236,0.228798,...,0.290155,-0.037637,0.012435,0.195928,0.110355,-0.091269,-0.026307,0.020561,0.489645,-0.063044
4,-0.622776,-1.300381,1.845811,-0.191194,0.013886,-1.246558,-0.029926,-0.086013,0.468155,0.261157,...,0.319756,-0.042134,-0.003155,0.240175,0.125228,-0.111572,-0.042119,0.016869,0.930811,-0.140841


In [88]:
test_origin.head()

,SITE_ID,DATE,N_TRANSPORTED_SITES
0,858,2019-04-10,10.0
1,858,2019-04-11,10.0
2,858,2019-04-12,10.0
3,858,2019-04-13,10.0
4,858,2019-04-14,10.0


In [89]:
test_df_concat=pd.concat([test_origin, X_pca_test], axis = 1)

In [90]:
test_df_concat.isnull().sum().sum()

0

In [122]:
####  Preprocessing of the test.csv in a exact process we did for the Train.csv 

Best_model = DecisionTreeClassifier(criterion= 'gini', max_depth= 10, min_samples_leaf= 10, min_samples_split= 20)
Best_model.fit(X, y)

y_2_pred = Best_model.predict(X_pca_test)



### Evaluation Metrics

In [123]:

test_df_with_target = test_df_concat
test_df_with_target['aircon_sum_target_next14d']= y_2_pred

#read submission

# submission_dataset = pd.read_csv(os.path.join(dataset_dir,'submission.csv'))
submission_dataset = pd.read_csv('submission.csv')

#keep relevant cols from test set

eval_df = test_df_with_target[['SITE_ID' , 'DATE' , 'N_TRANSPORTED_SITES' , 'aircon_sum_target_next14d' ]].copy()

#merge test and submission to have N_TRANSPORTED_SITES and target

submission_dataset = submission_dataset.merge(eval_df, on=['SITE_ID' , 'DATE' ])


In [124]:
submission_dataset['aircon_sum_target_next14d'].value_counts()


0    159472
1       728
Name: aircon_sum_target_next14d, dtype: int64

In [125]:

recalls = []

#compute daily performance

for date, daily_df in submission_dataset.groupby(['DATE']):

    daily_df=daily_df.copy() 
    
    daily_df['final_prediction' ] = 0

    daily_df['NORM_N_TRANSPORTED_SITES' ] = daily_df['N_TRANSPORTED_SITES' ]/daily_df['N_TRANSPORTED_SITES' ].max()

    daily_df['weighted_prediction' ] = 0.8* daily_df['prediction' ] + 0.2*daily_df['NORM_N_TRANSPORTED_SITES' ]

    daily_df = daily_df.sort_values(by=['weighted_prediction' ], ascending=False)

    daily_df = daily_df.reset_index(drop=True)

    daily_df.loc[0:10, 'final_prediction' ] = 1

    daily_recall = recall_score(y_true=daily_df['aircon_sum_target_next14d' ], y_pred=daily_df['final_prediction' ])


    recalls.append(daily_recall)

final_metric= np.mean(recalls)

print (final_metric)

/home/tara/anaconda3/envs/thesis/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1221: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 due to no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


0.016777777777777777


In [115]:
daily_df['aircon_sum_target_next14d' ].value_counts()

0    534
Name: aircon_sum_target_next14d, dtype: int64